In [1]:
from pathlib import *
import sqlalchemy as sa
import pandas as pd
import os
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})

database_management_sys = sa.create_engine('sqlite:///../../../../Desktop/rerun-data-no-long-run/se/main.db')
connect_dbms_to_db = database_management_sys.connect()

In [2]:
max_tournament_player_set = """
    SELECT MAX(tournament_player_set) FROM folk_theorem_experiment
"""
max_num_of_player_sets = pd.read_sql(max_tournament_player_set, connect_dbms_to_db)
maximum_player_set =max_num_of_player_sets["MAX(tournament_player_set)"][0]
maximum_player_set

96

In [3]:
player_set_collection = """
    SELECT * FROM folk_theorem_experiment
    WHERE warning_message = 'None'
    AND tournament_player_set = ?
"""

In [4]:
table_headings = pd.read_sql("folk_theorem_experiment", connect_dbms_to_db).columns.tolist()
table_headings

['experiment_number',
 'number_of_players',
 'tournament_player_set',
 'player_strategy_name',
 'is_long_run_time',
 'is_stochastic',
 'memory_depth_of_strategy',
 'prob_of_game_ending',
 'payoff_matrix',
 'num_of_repetitions',
 'num_of_equilibria',
 'nash_equilibria',
 'least_prob_of_defection',
 'greatest_prob_of_defection',
 'noise',
 'warning_message']

In [5]:
collect_relevant_data = connect_dbms_to_db.execute(player_set_collection, 70)
num_of_set_data = pd.DataFrame(collect_relevant_data.fetchall(), columns=table_headings)
num_of_set_data

,experiment_number,number_of_players,tournament_player_set,player_strategy_name,is_long_run_time,is_stochastic,memory_depth_of_strategy,prob_of_game_ending,payoff_matrix,num_of_repetitions,num_of_equilibria,nash_equilibria,least_prob_of_defection,greatest_prob_of_defection,noise,warning_message
0,77000,4,70,Hard Tit For Tat,0,0,3,0.001000,3. 1.04134168 1.01402163 0.9927317...,500,7,1. 0. 0. 0. ...,0.0,1.0,0.0,None
1,77000,4,70,Hard Prober,0,0,Inf,0.001000,3. 1.04134168 1.01402163 0.9927317...,500,7,1. 0. 0. 0. ...,0.0,1.0,0.0,None
2,77000,4,70,Hard Go By Majority: 5,0,0,5,0.001000,3. 1.04134168 1.01402163 0.9927317...,500,7,1. 0. 0. 0. ...,0.0,1.0,0.0,None
3,77000,4,70,Defector,0,0,0,0.001000,3. 1.04134168 1.01402163 0.9927317...,500,7,1. 0. 0. 0. ...,0.0,1.0,0.0,None
4,77001,4,70,Hard Tit For Tat,0,0,3,0.011081,3. 1.21876167 1.08945913 0.9479814...,500,7,1. 0. 0. 0. ...,0.0,1.0,0.0,None
5,77001,4,70,Hard Prober,0,0,Inf,0.011081,3. 1.21876167 1.08945913 0.9479814...,500,7,1. 0. 0. 0. ...,0.0,1.0,0.0,None
6,77001,4,70,Hard Go By Majority: 5,0,0,5,0.011081,3. 1.21876167 1.08945913 0.9479814...,500,7,1. 0. 0. 0. ...,0.0,1.0,0.0,None
7,77001,4,70,Defector,0,0,0,0.011081,3. 1.21876167 1.08945913 0.9479814...,500,7,1. 0. 0. 0. ...,0.0,1.0,0.0,None
8,77006,4,70,Hard Tit For Tat,0,0,3,0.061485,3. 1.43084902 1.30025513 0.8081757...,500,7,1. 0. 0. 0. ...,0.0,1.0,0.0,None
9,77006,4,70,Hard Prober,0,0,Inf,0.061485,3. 1.43084902 1.30025513 0.8081757...,500,7,1. 0. 0. 0. ...,0.0,1.0,0.0,None


In [11]:
degenerate_collection = """
    SELECT * FROM folk_theorem_experiment
    WHERE number_of_players = '4'
    AND tournament_player_set = '70'
    AND noise = '0.7'
"""

In [16]:
collect_degenerate_data = connect_dbms_to_db.execute(degenerate_collection)#, 4, 70, 0.7)
degenerate_data = pd.DataFrame(collect_degenerate_data.fetchall(), columns=table_headings)
degenerate_data


""


In [ ]:
#for num_of_sets in range(maximum_player_set):
for num_of_sets in [0, 24, 53, 70, 94]:
     collect_relevant_data = connect_dbms_to_db.execute(player_set_collection, num_of_sets)
     num_of_set_data = pd.DataFrame(collect_relevant_data.fetchall(), columns=table_headings)

     num_of_players = num_of_set_data["number_of_players"][0]

     for noise in list(num_of_set_data["noise"].drop_duplicates()):
          noise = round(noise, 1)

          p = Path("../../images/folk_thm/single_game/" + str(num_of_players) + "/" + str(num_of_sets) + "/" + str(noise) + "/")
          if p.exists() == False:
               p.mkdir(parents=True)
               print(str(p) + " was created.")
          else:
               print(str(p) + " already exists.")

          markdown_path = p / "README.md"
          strategy_table = pd.DataFrame(num_of_set_data[["player_strategy_name", "is_long_run_time", "is_stochastic", "memory_depth_of_strategy"]].iloc[list(range(num_of_players))])                   
          README = open(str(markdown_path), "w")
          README.write("# Single Game Plot \n" + str(strategy_table) + "\n ## Noise = " + str(noise))
          README.close()
          
          specific_noise_data = num_of_set_data[num_of_set_data["noise"] == noise]
          
          if len(specific_noise_data) > 0:
               plot_path = p / "main.pdf"
               graph = plt.figure()
               axes = graph.add_subplot(1, 1, 1)
               axes.set_xlabel("$p =$ the probability of the game ending" )
               axes.set_ylabel("probability of defection in equilibria")
               axes.plot(specific_noise_data["prob_of_game_ending"], specific_noise_data["least_prob_of_defection"], "r", label = "least prob of defection")
               axes.plot(specific_noise_data["prob_of_game_ending"], specific_noise_data["greatest_prob_of_defection"], "y--", label = "greatest prob of defection")
               axes.legend()
               graph.savefig(str(plot_path)) 
               plt.close()
          else:

               

               text_path = p / "could-be-degenerate.txt"
               degenerate_file = open(str(text_path), "w")
               degenerate_file.write("This particular game could be degenerate.\n" + "WARNINGS PRODUCED: " + ???? + "\nPayoff matrix obtained: " + ????)
               degenerate_file.close()

In [ ]:
dbms = sa.create_engine('sqlite:///../Database-code/data/vertex_enumeration/main.db')
connect_to_db = database_management_sys.connect()